In [ ]:
#http://www.ritchieng.com/machine-learning/deep-learning/tensorflow/deep-neural-nets/
#http://ischlag.github.io/2016/06/04/how-to-use-tensorboard/
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
tf.set_random_seed(42)

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('precision', 18)

In [ ]:
es = pd.read_csv("../vectors/es.node2vec.embeddings", delimiter = " ", skiprows = 1, header = None)
na = pd.read_csv("../vectors/na.node2vec.embeddings", delimiter = " ", skiprows = 1, header = None)

In [ ]:
es.head()

In [ ]:
na.head()

In [ ]:
es_vectores = np.array(es[:1900].loc[:,1::]).astype(np.float32)
na_vectores = np.array(na[:1900].loc[:,1::]).astype(np.float32)

In [ ]:
es_vectores

In [ ]:
na_vectores

In [ ]:
es_palabras = es[0]
na_palabras = na[0]

In [ ]:
print("shape es:",es_vectores.shape,"\nshape na:",na_vectores.shape)

# Hyperparameters

In [ ]:
# Hyperparameters
LEARNING_RATE = 0.1
EPOCHS = 10
BATCH_SIZE = 100

NODES_INPUT = es_vectores[0].size
NODES_H1 = 100
NODES_H2 = 100
NODES_OUPUT = na_vectores[0].size
INSTANCES = es_vectores.__len__()
NUM_STEPS = 5000

In [ ]:
with tf.name_scope("inputs"):
    X = tf.placeholder(shape=[None,NODES_INPUT], dtype=tf.float32,name='X')
    y = tf.placeholder(shape=[None,NODES_OUPUT], dtype=tf.float32,name='y')
    print("X:",X.shape,"y:",y.shape)

In [ ]:
# Capas ocultas (weights & bias)
with tf.name_scope("hidden_layer1"):
    hidden_layer1 = {"W1":tf.Variable(tf.truncated_normal([NODES_INPUT,NODES_H1],stddev=0.01),name = 'W1'),
                 "b1":tf.constant(0.1,shape=[NODES_H1], name = 'b1')}
with tf.name_scope("hidden_layer2"):
    hidden_layer2 = {"W2":tf.Variable(tf.truncated_normal([NODES_H1,NODES_H2],stddev=0.01),name = 'W2'),
                     "b2":tf.constant(0.1,shape=[NODES_H2], name = 'b2')}

# Capa de salida
with tf.name_scope("output_layer1"):
    output_layer = {"W2":tf.Variable(tf.truncated_normal([NODES_H1,NODES_OUPUT],stddev=0.01),name = 'W_output'),
                    "b2":tf.constant(0.1,shape=[NODES_OUPUT], name = 'b_output')}

In [ ]:
print(hidden_layer1)

In [ ]:
# Calcular la salida de la 1er capa oculta
# h(x) = x*w + bias
hidden_layer1_output = tf.add(tf.matmul(es_vectores, hidden_layer1["W1"]), hidden_layer1["b1"],name="hidden1")
tf.summary.scalar('pre_activations1', hidden_layer1_output)
# Función de activación usando ReLU
with tf.name_scope("relu_layer1"):
    hidden_layer1_output = tf.nn.relu(hidden_layer1_output,name="actfhidden1")


In [ ]:
# Calcular la salida de la 2da capa oculta
# h(x) = x*w + bias
hidden_layer2_output = tf.add(tf.matmul(hidden_layer1_output, hidden_layer2["W2"]), hidden_layer2["b2"],name="hidden2")
tf.summary.scalar('pre_activations2', hidden_layer2_output)
# Función de activación usando ReLU
with tf.name_scope("relu_layer2"):
    hidden_layer2_output = tf.nn.relu(hidden_layer2_output,name="actfhidden2")

In [ ]:
# calcular la salida la NN
output = tf.add(tf.matmul(hidden_layer2_output, output_layer["W2"]),output_layer["b2"],name="output")
# Función de activación usando softmax
with tf.name_scope("softmax"):
    nah_predicted = tf.nn.softmax(output,name="actfoutput")
print(nah_predicted.shape,y.shape)

In [ ]:
# Función de error (Mean Square Error)
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = nah_predicted))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.squared_difference(nah_predicted,y),name="loss_f")
#loss = tf.reduce_sum((nah_predicted - y) ** 2)

In [ ]:
# optimiser, 
with tf.name_scope("optimiser"):
    optimiser = tf.train.GradientDescentOptimizer(learning_rate = LEARNING_RATE,name="GradientDescent").minimize(loss,name="loss")

In [ ]:
tf.summary.scalar("loss",loss)
summary_op = tf.summary.merge_all()


In [ ]:
#Session
sess = tf.Session()
# Initialize variables
init = tf.global_variables_initializer()
sess.run(init)
#writer = tf.summary.FileWriter("./logs/NN",sess.graph)
writer = tf.summary.FileWriter("./logs/NN", sess.graph)

In [ ]:
tmp_hidden_layer1,tmp_hidden_layer2 = sess.run(hidden_layer1),sess.run(hidden_layer2)

In [ ]:
for step in range(3000):
    '''
    offset = (step * BATCH_SIZE) % (es_vectores.shape[0] - BATCH_SIZE)
    batch_data = es_vectores[offset:(offset + BATCH_SIZE), :]
    batch_target = na_vectores[offset:(offset + BATCH_SIZE), :]
    '''
    _, _loss = sess.run([optimiser,loss],feed_dict={X:es_vectores,y:na_vectores})
    
    if (step % 100) == 0:
        print(_loss)

In [ ]:
print(sess.run(hidden_layer1))

In [ ]:

tmp_hidden_layer1

In [ ]:
sess.run(hidden_layer2)

In [ ]:
import tensorflow as tf

# reset everything to rerun in jupyter
tf.reset_default_graph()

# config
batch_size = 100
learning_rate = 0.5
training_epochs = 5
logs_path = "/tmp/mnist/2"

# load mnist data set
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# input images
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")

# model parameters will change during training so we use tf.Variable
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))

# bias
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

# implement model
with tf.name_scope("softmax"):
    # y is our prediction
    y = tf.nn.softmax(tf.matmul(x,W) + b)

# specify cost function
with tf.name_scope('cross_entropy'):
    # this is our cost
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# specify optimizer
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
    # Accuracy
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# create a summary for our cost and accuracy
tf.scalar_summary("cost", cross_entropy)
tf.scalar_summary("accuracy", accuracy)

# merge all summaries into a single "operation" which we can execute in a session 
summary_op = tf.merge_all_summaries()

with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())

    # create log writer object
    writer = tf.train.SummaryWriter("./logs/NN", graph=tf.get_default_graph())
        
    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(mnist.train.num_examples/batch_size)
        
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # perform the operations we defined earlier on batch
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
            
            # write log
            writer.add_summary(summary, epoch * batch_count + i)
            
        if epoch % 5 == 0: 
            print "Epoch: ", epoch 
    print "Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print "done"